In [335]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


In [336]:
train_data = pd.read_csv('house-prices-advanced-regression-techniques/train.csv', index_col=0)
test_data = pd.read_csv('house-prices-advanced-regression-techniques/test.csv', index_col=0)

In [232]:
train_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [233]:
test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml


In [286]:
df = pd.concat([train_data , test_data])
df = df.drop(['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'FireplaceQu'], axis=1)
df = pd.get_dummies(df)


In [287]:
train = df[df.index.isin(train_data.index)]
test = df.drop(train_data.index)

In [288]:
test = df.drop(train_data.index)

In [289]:
print(train.shape, test.shape, train_data.shape)

(1460, 271) (1459, 271) (1460, 80)


In [308]:
X = train

In [240]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(X.isnull().sum())

MSSubClass                 0
LotFrontage              259
LotArea                    0
OverallQual                0
OverallCond                0
YearBuilt                  0
YearRemodAdd               0
MasVnrArea                 8
BsmtFinSF1                 0
BsmtFinSF2                 0
BsmtUnfSF                  0
TotalBsmtSF                0
1stFlrSF                   0
2ndFlrSF                   0
LowQualFinSF               0
GrLivArea                  0
BsmtFullBath               0
BsmtHalfBath               0
FullBath                   0
HalfBath                   0
BedroomAbvGr               0
KitchenAbvGr               0
TotRmsAbvGrd               0
Fireplaces                 0
GarageYrBlt               81
GarageCars                 0
GarageArea                 0
WoodDeckSF                 0
OpenPorchSF                0
EnclosedPorch              0
3SsnPorch                  0
ScreenPorch                0
PoolArea                   0
MiscVal                    0
MoSold        

In [44]:
# X.FireplaceQu.unique()

array([nan, 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

In [200]:
X.LotFrontage.describe()

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [309]:
X = X.fillna({"LotFrontage": X.LotFrontage.median()})

In [310]:
X.shape

(1460, 271)

In [294]:
X = X.fillna(0)

In [311]:
X = X.dropna(axis=0)

In [85]:
# X = X.dropna(subset=['GarageType', 'BsmtExposure','MasVnrType', 'BsmtFinType2', 'Electrical'], axis=0)

In [66]:
# X = X[X.GarageType.notna()]

In [67]:
# X = X[X.BsmtExposure.notna()]

In [68]:
X.loc[(X.BsmtQual != "Gd") & ( X.BsmtQual != "TA") & (X.BsmtQual != "Ex") & (X.BsmtQual != "Fa" )]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,


In [181]:
# X_fixed = pd.get_dummies(X)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1457,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,1,0,0,0,0,1,0
1458,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,1,0,0,0,0,1,0


In [312]:
X.shape

(1371, 271)

In [313]:
y = X.SalePrice
X = X.drop('SalePrice', axis=1)

In [314]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [315]:
X_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
459,70,69.0,5100,8,7,1925,1996,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
629,60,70.0,11606,5,5,1969,1969,192.0,650.0,0.0,...,0,0,0,1,0,0,0,1,0,0
319,60,90.0,9900,7,5,1993,1993,256.0,987.0,0.0,...,0,0,0,1,0,0,0,0,1,0
200,20,76.0,9591,8,5,2004,2005,262.0,1088.0,0.0,...,0,0,0,1,0,0,0,0,1,0
264,50,50.0,5500,5,7,1929,2001,0.0,234.0,692.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,60,69.0,14803,6,5,1971,1971,252.0,416.0,0.0,...,0,0,0,1,0,0,0,0,1,0
755,20,61.0,7930,6,8,1969,2005,0.0,439.0,472.0,...,0,0,0,1,0,0,0,0,1,0
700,120,59.0,4282,7,5,2004,2004,0.0,16.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [316]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
prediction = rf.predict(X_test)

### Choosing the best parametres 

In [337]:
clf_rf = RandomForestClassifier()

In [349]:
parametrs = {'n_estimators': [10,15, 22, 24, 26], 'max_depth' : [7, 10, 13, 15], 
             'min_samples_leaf': range(1, 5)}

In [350]:
grid_search_cv_clf = GridSearchCV(clf_rf, parametrs, cv=5)

In [346]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1280,50,60.0,7500,4,4,1920,1950,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1308,20,60.0,8072,5,5,1994,1995,0.0,746.0,0.0,...,0,0,0,1,0,0,0,0,1,0
243,50,63.0,5000,5,4,1900,1950,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1002,30,60.0,5400,5,6,1920,1950,0.0,0.0,0.0,...,0,0,0,1,1,0,0,0,0,0
296,80,37.0,7937,6,6,1984,1984,0.0,819.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,20,64.0,10475,8,5,2008,2008,72.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1204,20,75.0,9750,7,5,2000,2001,171.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1380,80,73.0,9735,5,5,2006,2007,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [347]:
y_train

Id
1280     68400.0
1308    138000.0
243      79000.0
1002     86000.0
296     142500.0
          ...   
1167    245350.0
1204    213000.0
1380    167500.0
913      88000.0
1200    148000.0
Name: SalePrice, Length: 918, dtype: float64

In [351]:
grid_search_cv_clf.fit(X_train, y_train)

/Users/yulialysenko/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [353]:
best_clf = grid_search_cv_clf.best_estimator_

In [354]:
best_clf.score(X_test, y_test)

0.019867549668874173

In [355]:
predict = best_clf.predict(test)

In [103]:
from sklearn import metrics


In [318]:
metrics.accuracy_score(y_test, prediction)

0.006622516556291391

In [105]:
y_test

Id
833     237000
966     178900
970     140000
1412    140000
285     179200
         ...  
704     140000
662     402000
799     485000
352     190000
522     150000
Name: SalePrice, Length: 442, dtype: int64

In [300]:
test = test.fillna({"LotFrontage": test.LotFrontage.median()})

In [301]:
test = test.drop('SalePrice', axis=1)

In [302]:
test = test.fillna(0)

In [319]:
test.shape

(1459, 270)

In [304]:
X.shape

(1460, 270)

In [228]:
test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,20,80.0,13384,5,5,1969,1979,194.0,119.0,344.0,...,0,0,0,1,0,0,0,0,1,0
2913,160,21.0,1533,4,5,1970,1970,0.0,408.0,0.0,...,0,0,0,1,1,0,0,0,0,0
2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,0,0,0,1,1,0,0,0,0,0


In [166]:
# test_data = test_data.drop(['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'FireplaceQu'], axis=1)

In [167]:
# test_data = test_data.fillna({"LotFrontage": test_data.LotFrontage.median()})

In [320]:
y_pred = rf.predict(test)

In [271]:
sample = pd.read_csv('house-prices-advanced-regression-techniques/sample_submission.csv')

In [272]:
sample

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [356]:
test_data = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')

In [357]:
test_data['SalePrice'] = y_pred

In [323]:
y_pred.shape

(1459,)

In [358]:
pred_df = test_data[['Id', 'SalePrice']]
pred_df.set_index('Id', inplace=True)

In [359]:
pred_df

,SalePrice
Id,
1461,145000.0
1462,106500.0
1463,189000.0
1464,140000.0
1465,171750.0
...,...
2915,88000.0
2916,88000.0
2917,160000.0


In [360]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
pred_df.to_csv('out.zip',
          compression=compression_opts)